# Unicode文字列

## はじめに

自然言語モデルは、しばしば異なる文字セットを使った異なる言語を扱う。
Unicodeは、ほぼすべての言語れ文字表示に使われている標準的なエンコードの仕組み。各文字は、`0`から`0x10FFFF`までの一意の整数の
[コードポイント(符号点))](https://ja.wikipedia.org/wiki/%E7%AC%A6%E5%8F%B7%E7%82%B9)
を使ってエンコードされる。1つのUnicode文字列は、ゼロ個以上のコードポイントのシーケンスになっている。

このチュートリアルでは、TensorFlowでのUnicode文字列の表現方法と、どうやってUnicodeで標準的な文字列操作と同様の操作を行うかについて示す。
また、スクリプト検出に基づいてUnicode文字列をトークンに分解する。

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

## tf.stringデータ型

標準的なTensorFlowの`tf.string`型は、バイト列のテンソルを作る。また、Unicode文字列はデフォルトではutf-8でエンコードされる。

In [3]:
tf.constant(u"Thanks 😊")

<tf.Tensor: shape=(), dtype=string, numpy=b'Thanks \xf0\x9f\x98\x8a'>

バイト列が最小限の単位として扱われるため、`tf.string`型のテンソルは可変長のバイト列を保持できる。
また、文字列長はテンソルの次元には含まれない。

注：Pythonを使って文字列を構成する時、v2.x系とv3.x系ではUnicodeの扱いが異なる。
v2.x系では、Unicode文字列は上記のようにプレフィックス"u"で明示する。
v3.x系では、デフォルトでUnicodeとしてエンコードされる。

In [6]:
tf.constant([u"You're]", u"welcome!"]).shape

TensorShape([2])

## Unicode表現

TensorFlowでのUnicode文字列表現は、2つの標準的な方法がある：

- `string`スカラー - コードポイントのシーケンスは既知の文字列符号化方式でエンコードされる。
- `int32`ベクトル - 各文字には単一のコードポイントが入る。

例えば、以下3つはすべてUnicode文字列`语言処理`(中国語で「言語処理」を意味する)を表す。

In [7]:
# Unicode文字列。UTF-8にエンコードされた文字列スカラーとして表される。
text_utf8 = tf.constant(u"语言処理")
text_utf8

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\x87\xa6\xe7\x90\x86'>

In [11]:
# Unicode文字列。UTF-16-BEにエンコードされた文字列スカラーとして表される。
text_utf16 = tf.constant(u"语言処理".encode("UTF-16-BE"))
text_utf16

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Q\xe6t\x06'>

In [13]:
# Unicode文字列。Unicodeコードポイントのベクトルとして表される。
text_chars = tf.constant([ord(char) for char in u"语言処理"])
text_chars

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 20966, 29702])>

### Unicode表現間の変換

TensorFlowでは、これらの異なるUnicode表現間で変換する方法を用意している。

- `tf.strings.unicode_decode`: エンコードされた文字列スカラーを、コードポイントのベクトルに変換する。
- `tf.strings.unicode_encode`: コードポイントのベクトルを、エンコードされた文字列スカラーに変換する。
- `tf.strings.unicode_transcode`: エンコードされた文字列スカラーを、別の文字コードに再エンコードする。

In [14]:
tf.strings.unicode_decode(text_utf8, input_encoding="UTF-8")

<tf.Tensor: shape=(4,), dtype=int32, numpy=array([35821, 35328, 20966, 29702])>

In [15]:
tf.strings.unicode_encode(text_chars, output_encoding="UTF-8")

<tf.Tensor: shape=(), dtype=string, numpy=b'\xe8\xaf\xad\xe8\xa8\x80\xe5\x87\xa6\xe7\x90\x86'>

In [17]:
tf.strings.unicode_transcode(text_utf8, input_encoding="UTF-8", output_encoding="UTF-16-BE")

<tf.Tensor: shape=(), dtype=string, numpy=b'\x8b\xed\x8a\x00Q\xe6t\x06'>

### バッチの次元

複数の文字列をデコードする場合、各文字列の文字数が等しくない場合がある。
返される結果は`tf.RaggedTensor`であり、最も内側の次元の長さは各文字列の文字数によって異なる。

In [18]:
# Unicode文字列のバッチ。それぞれが、UTF8にエンコードされた文字列として表される
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo',  u'What is the weather tomorrow',  u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
  print(sentence_chars)

[104, 195, 108, 108, 111]
[87, 104, 97, 116, 32, 105, 115, 32, 116, 104, 101, 32, 119, 101, 97, 116, 104, 101, 114, 32, 116, 111, 109, 111, 114, 114, 111, 119]
[71, 246, 246, 100, 110, 105, 103, 104, 116]
[128522]


このデータの処理方法には3つの選択肢がある。

1. この`tf.RaggedTensor`を直接使用する
2. `tf.RaggedTensor.to_tensor`メソッドを使ってパディングを追加した密な`tf.Tensor`に変換する
3. `tf.RaggedTensor.to_sparse`メソッドを使って`tf.SparseTensor`に変換する

In [19]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded)

tf.Tensor(
[[   104    195    108    108    111     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [    87    104     97    116     32    105    115     32    116    104
     101     32    119    101     97    116    104    101    114     32
     116    111    109    111    114    114    111    119]
 [    71    246    246    100    110    105    103    104    116     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]
 [128522     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1     -1     -1
      -1     -1     -1     -1     -1     -1     -1     -1]], shape=(4, 28), dtype=int32)


In [21]:
batch_chars_sparse = batch_chars_ragged.to_sparse()
print(batch_chars_sparse)

SparseTensor(indices=tf.Tensor(
[[ 0  0]
 [ 0  1]
 [ 0  2]
 [ 0  3]
 [ 0  4]
 [ 1  0]
 [ 1  1]
 [ 1  2]
 [ 1  3]
 [ 1  4]
 [ 1  5]
 [ 1  6]
 [ 1  7]
 [ 1  8]
 [ 1  9]
 [ 1 10]
 [ 1 11]
 [ 1 12]
 [ 1 13]
 [ 1 14]
 [ 1 15]
 [ 1 16]
 [ 1 17]
 [ 1 18]
 [ 1 19]
 [ 1 20]
 [ 1 21]
 [ 1 22]
 [ 1 23]
 [ 1 24]
 [ 1 25]
 [ 1 26]
 [ 1 27]
 [ 2  0]
 [ 2  1]
 [ 2  2]
 [ 2  3]
 [ 2  4]
 [ 2  5]
 [ 2  6]
 [ 2  7]
 [ 2  8]
 [ 3  0]], shape=(43, 2), dtype=int64), values=tf.Tensor(
[   104    195    108    108    111     87    104     97    116     32
    105    115     32    116    104    101     32    119    101     97
    116    104    101    114     32    116    111    109    111    114
    114    111    119     71    246    246    100    110    105    103
    104    116 128522], shape=(43,), dtype=int32), dense_shape=tf.Tensor([ 4 28], shape=(2,), dtype=int64))


同じ長さの複数の文字列をエンコードする場合、`tf.Tensor`を入力値として使用できる。

In [22]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [99, 111, 119]], output_encoding="UTF-8")

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'cat', b'dog', b'cow'], dtype=object)>

可変長の複数の文字列をエンコードする場合、`tf.RaggedTensor`を入力値として使用する必要がある。

In [24]:
# batch_chars_raggedはtf.RaggedTensor
print(type(batch_chars_ragged))
tf.strings.unicode_encode(batch_chars_ragged, output_encoding="UTF-8")

<class 'tensorflow.python.ops.ragged.ragged_tensor.RaggedTensor'>


<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

パディングされた、あるいはスパースな複数の文字列を含むテンソルがある場合は、`unicode_encode`を呼び出す前に`tf.RaggedTensor`に変換する。

In [25]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse), output_encoding="UTF-8"
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

In [26]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1), output_encoding="UTF-8"
)

<tf.Tensor: shape=(4,), dtype=string, numpy=
array([b'h\xc3\x83llo', b'What is the weather tomorrow',
       b'G\xc3\xb6\xc3\xb6dnight', b'\xf0\x9f\x98\x8a'], dtype=object)>

## Unicode操作

### 文字列長

`tf.strings.length`には、文字列長をどう計算するか示す`unit`パラメーターが使える。
`unit`のデフォルトは`"BYTE"`だが、`"UTF-8_CHAR"`や`"UTF-16_CHAR"`などの他の値に設定して、エンコードされた`string`文字列のUnicodeコードポイントの数を決めることができる。

In [27]:
# 最後の絵文字は、UTF8で4バイトを占めることに注意する。
thanks = u"Thanks 😊".encode("UTF-8")
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit="UTF8_CHAR").numpy()
print("{} bytes; {} UTF-8 characters".format(num_bytes, num_chars))

11 bytes; 8 UTF-8 characters


### 部分文字列

同様に、`tf.strings.substr`では、`unit`パラメータを使い、かつ`pos`および`len`パラメーターを指定することで、オフセットの種類を決めることができる。

In [28]:
# デフォルト：unit="BYTE". len=1 の場合、1バイトを返す
tf.strings.substr(thanks, pos=7, len=1).numpy()

b'\xf0'

In [29]:
# unit = "UTF8_CHAR"を指定すると、単一の文字（この場合は4バイト）が返される
print(tf.strings.substr(thanks, pos=7, len=1, unit="UTF8_CHAR").numpy)

<bound method _EagerTensorBase.numpy of <tf.Tensor: shape=(), dtype=string, numpy=b'\xf0\x9f\x98\x8a'>>


### Unicode部文字列を分割する

`tf.strings.unicode_split`は、Unicode文字列を個々の文字に分割する。

In [30]:
tf.strings.unicode_split(thanks, "UTF-8").numpy()

array([b'T', b'h', b'a', b'n', b'k', b's', b' ', b'\xf0\x9f\x98\x8a'],
      dtype=object)

### 文字のバイトオフセット

`tf.strings.unicode_decode`によって生成された文字テンソルを元の文字列に戻すには、各文字の開始位置のオフセットを知ることが役立つ。

`tf.strings.unicode_decode_with_offset`メソッドは`unicode_decode`に似ているが、各文字の開始オフセットを含む2番目のテンソルを返す点が異なる。

In [31]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u"🎈🎉🎊", 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
  print("At byte offset {}: codepoint {}".format(offset, codepoint))

At byte offset 0: codepoint 127880
At byte offset 4: codepoint 127881
At byte offset 8: codepoint 127882


## Unicodeスクリプト

各Unicodeコードポイントは、スクリプトとして知られる単一のコードポイント集合に属している。
文字スクリプトは、その文字がどの言語なのかを判断するのに役立つ。
例えば、「Б」がキリル文字であることがわかれば、その文字を含むテキストはロシア語やウクライナ語などのスラブ言語である可能性が高いことがわかる。

TensorFlowは、あるコードポイントがどのスクリプトかを返す`tf.strings.unicode_script`を提供している。
戻り値のスクリプトコードは、International Components for Unicode (ICU)の`UScriptCode`に対応する`int32`値になる。


In [34]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

[17  8]


`tf.strings.unicode_script`は、多次元のコードポイントの`tf.Tensor`や`tf.RaddedTensor`にも適用できる。

In [35]:
print(tf.strings.unicode_script(batch_chars_ragged))

<tf.RaggedTensor [[25, 25, 25, 25, 25], [25, 25, 25, 25, 0, 25, 25, 0, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 0, 25, 25, 25, 25, 25, 25, 25, 25], [25, 25, 25, 25, 25, 25, 25, 25, 25], [0]]>


## 例：シンプルなセグメンテーション

セグメンテーションは、テキストを単語のような粒度に分割するタスク。
これは、スペース文字を使用して単語を区切れる場合には簡単に行えるが、一部の言語（中国語や日本語など）はスペースを使わないし、
また、一部の言語（ドイツ語など）には、意味を解析するため分ける必要がある、単語を結合した長い複合語がある。
Webテキストでは「NY株価」（ニューヨーク株価）のように、異なる言語とスクリプトがしばしば混在している。

単語の境界を推定してスクリプトを変更することにより、（MLモデルを実装せずに）非常に大まかなセグメンテーションを実行できる。
これは、上記の「NY株価」の例のような文字列に対して機能する。
様々な言語のスペース文字はすべて、実際のテキストとは異なる特別なスクリプトコードであるUSCRIPT_COMMONとして分類されるため、スペースを使用するほとんどの言語で機能する。


In [36]:
# dtype: string; shape: [num_sentences]
# 処理する文章。この行を編集して、さまざまな入力を試してみること
sentence_texts = [u'Hello, world.', u'世界こんにちは']

最初に、文章を文字ごとのコードポイントにデコードし、それから各文字のスクリプトコード（識別子）を調べる。

In [39]:
# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j] は、i番目の文のj番目の文字のコードポイント
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_scripts[i, j] は、i番目の文のj番目の文字のスクリプトコード
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

<tf.RaggedTensor [[72, 101, 108, 108, 111, 44, 32, 119, 111, 114, 108, 100, 46], [19990, 30028, 12371, 12435, 12395, 12385, 12399]]>
<tf.RaggedTensor [[25, 25, 25, 25, 25, 0, 0, 25, 25, 25, 25, 25, 0], [17, 17, 20, 20, 20, 20, 20]]>


次に、これらのスクリプトコードを使って、単語の境界を追加すべき場所を決める。
前の文字とスクリプトコードが異なるそれぞれの文字の先頭に、単語の境界を追加する。

In [44]:
# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j] は、i番目の文のj番目の文字が単語の始まりである場合にTrue
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True), #1文字目は必ずTrue
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])], # スクリプトが異なるかどうか比較 第一引数は1index始まり 第二引数は0index始まりにすることで、1文字前との比較を実現
    axis=1)
print(sentence_char_starts_word)

# dtype: int64; shape: [num_words]
#
# word_starts[i] は、i番目の単語の始まりである文字のインデックス
# （すべての文がフラット化された文字リスト）
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

<tf.RaggedTensor [[True, False, False, False, False, True, False, True, False, False, False, False, True], [True, False, True, False, False, False, False]]>
tf.Tensor([ 0  5  7 12 13 15], shape=(6,), dtype=int64)


そして、これらの目印（開始オフセット）を使って、各単語ごとの文字リストを含む`RaggedTensor`を作成する。

In [46]:
# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j] は、i番目の単語のj番目の文字のコードポイント
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts)
print(word_char_codepoint)

<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46], [19990, 30028], [12371, 12435, 12395, 12385, 12399]]>


最後に、`RaggedTensor`をコードポイント単位でセグメント化して、文章に戻す。


In [51]:
# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i] は、i番目の文の単語数
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# 'Hello, world.'は4単語分割、'世界こんにちは'は2単語分割できる、という結果が返ってくる。
print(sentence_num_words)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k] は、i番目の文のj番目の単語のk番目の文字のコードポイント
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(word_char_codepoint) # Before
print(sentence_word_char_codepoint) # After

tf.Tensor([4 2], shape=(2,), dtype=int64)
<tf.RaggedTensor [[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46], [19990, 30028], [12371, 12435, 12395, 12385, 12399]]>
<tf.RaggedTensor [[[72, 101, 108, 108, 111], [44, 32], [119, 111, 114, 108, 100], [46]], [[19990, 30028], [12371, 12435, 12395, 12385, 12399]]]>


最終的な結果を見やすくするために、UTF-8文字列にエンコードする。

In [52]:
tf.strings.unicode_encode(sentence_word_char_codepoint, "UTF-8").to_list()

[[b'Hello', b', ', b'world', b'.'],
 [b'\xe4\xb8\x96\xe7\x95\x8c',
  b'\xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf']]